## **NOTE - CELLS 1-4 CONTAIN CODE THAT HELPS IMPORT THE DATASET FROM GOOGLE DRIVE, THE CODE FOR DEVELOPING THE MODEL STARTS FROM CELL 5**

In [0]:
## Google Drive Acknowledgement and mounting
from google.colab import drive
drive.mount('/content/drive')

In [0]:
! pip install pydrive
import pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
def get_file_from_drive(file_name):
  file_list = drive.ListFile({'q': "'1nZC09V7krEuUtRTOZkx-IB87LN2Jv46i' in parents and trashed=false"}).GetList()
  for file in file_list:
    if(file['title'] == file_name):
      return file['id']
    
def upload_data_system(file_name):
  downloaded = drive.CreateFile({'id': get_file_from_drive(file_name)})
  downloaded.GetContentFile(file_name)

In [0]:
def import_data_to_runtime():
  # importing the data from google to drive to the hosted runtime
  print('Importing Data to Runtime...')
  upload_data_system('train_images.npy')
  upload_data_system('train_labels.npy')
  upload_data_system('coord.npy')
#   upload_data_system('train_images_18.npy')
#   upload_data_system('train_labels_18.npy')
#   upload_data_system('train_images_17.npy')
#   upload_data_system('train_labels_17.npy')

In [0]:
# import the necessary packages
from keras_applications.resnet import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import regularizers
from sklearn.model_selection import train_test_split
import keras.backend, keras.layers, keras.models, keras.utils
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [0]:
def extract_foreground(image, rect):
  mask = np.zeros(img.shape[:2], np.uint8)

  bgdModel = np.zeros((1, 65), np.float64)
  fgdModel = np.zeros((1, 65), np.float64)

  rect = (77, 0, 98, 224)

  print("[INFO] Extracting Foreground...")
  cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 10, cv2.GC_INIT_WITH_RECT)

  mask2 = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')
  
  return img = img * mask2[:, :, np.newaxis]

In [0]:
def preprocess_images():
  # load the images into memory
  train_images = np.load('train_images.npy')
  coord = np.load('coord.npy')
  
  train_images_processed = []
  count = 1
  # resize the images to fit the input shape
  # of resnet (224, 224)
  for (i, image) in enumerate(train_images):
    resized_image = cv2.resize(image, (224, 224), cv2.INTER_AREA)
    foreground_extracted_image = extract_foreground(image, coord[i])
    train_images_processes.append(resized_image)
    print("Resized image {}/{}".format(count, train_images.shape[0]))
    count = count + 1
  
  np.save('train_images_preprocessed.npy', np.array(train_images_preprocessed))
  

In [0]:
def build_model():  
  base_model = ResNet50(include_top = False, backend = keras.backend, layers = keras.layers, models = keras.models, utils = keras.utils, input_shape = (224, 224, 3))
  
  model = Sequential()
  model.add(base_model)
  model.add(Flatten())
  model.add(Dense(units = 32, activation = 'relu', kernel_initializer = 'he_normal'))
  model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer = 'he_normal'))
  
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  
  print(model.summary())
  
  return model

In [0]:
def load_data():
  # load the data into memory and split the dataset into train and test splits
  train_images = np.load('train_images_preprocessed.npy')
  train_labels = np.load('train_labels.npy')

  x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels, test_size = 0.15, random_state = 100)
  
  return x_train, x_test, y_train, y_test

In [0]:
import_data_to_runtime()
preprocess_images()
model = build_model()
x_train, x_test, y_train, y_test = load_data()
History = model.fit(x_train, y_train, epochs = 50, batch_size = 32, validation_data = (x_test, y_test))

In [0]:
def wrong_predictions():
  # function returns indices of images that the model predicted wrong
  y_test2 = np.reshape(y_test, (y_test.shape[0], 1))
  y_pred = model.predict(x_test)
  result = np.array(y_pred >= 0.5)
  indices = np.where(result != y_test2)
  
  return indices

In [0]:
def plot_wrong_predictions():
  indices = wrong_predictions()

  count = 1
  for i in indices[0]:
    plt.subplot(3, 3, count)
    plt.imshow(x_test[i].astype(np.float32))
    count = count + 1

In [0]:
model.save('RS50w32_1.h5')

ModuleNotFoundError: No module named 'keras'